# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.127077e+02     1.684008e+00
 * time: 0.12131595611572266
     1     1.095008e+01     9.695978e-01
 * time: 0.908174991607666
     2    -1.154446e+01     1.119640e+00
 * time: 1.0407299995422363
     3    -3.394052e+01     8.384228e-01
 * time: 1.1934537887573242
     4    -4.741654e+01     5.818818e-01
 * time: 1.3354439735412598
     5    -5.690203e+01     2.242481e-01
 * time: 1.4754638671875
     6    -5.985299e+01     1.448206e-01
 * time: 1.5785880088806152
     7    -6.100937e+01     5.456235e-02
 * time: 1.6891229152679443
     8    -6.145553e+01     5.738248e-02
 * time: 1.7928388118743896
     9    -6.169720e+01     4.029715e-02
 * time: 1.9028728008270264
    10    -6.189767e+01     2.244481e-02
 * time: 2.0132949352264404
    11    -6.202819e+01     1.776429e-02
 * time: 2.1167848110198975
    12    -6.208974e+01     1.466616e-02
 * time: 2.2264299392700195
    13    -6.213432e+01     1.459110e-02
 * time: 2.3292930126190

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671073
    AtomicLocal         -18.8557726
    AtomicNonlocal      14.8522682
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485383 
    Xc                  -19.3336824

    total               -62.261666461775
